# **VALORES S.A.**

La firma Valores S.A. está estudiando la posibilidad de realizar algunas inversiones. La firma tiene la posibilidad de invertir en diez proyectos diferentes, los cuales son denotados en la tabla siguiente. Cada uno de los proyectos tiene una duración de un año. Estos proyectos difieren entre sí en el rendimiento neto y en el desembolso de capital que necesitan. El objetivo de la firma es maximizar el rendimiento neto total al final del año. El capital total disponible para afrontar estos proyectos es de 40 u.m. En cada proyecto se puede invertir una única vez. Solo se puede invertir en máximo 7 proyectos. Adicionalmente, existen un conjunto de restricciones impuestas por las autoridades las cuales se muestran a continuación y se deben cumplir simultáneamente:

- No se puede invertir al tiempo en los proyectos 3, 4 y 7.
- El proyecto 3 debe elegirse si se elige el 6.
- El proyecto 8 solo puede elegirse si se invierte en las opciones 10 o 4.
- El proyecto 4 es complementario del 10, luego se aceptan o se rechazan conjuntamente.

| PROYECTOS                                    | P1  | P2  | P3  | P4  | P5  | P6  | P7  | P8  | P9  | P10 |
| -------------------------------------------- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| **Inversión (u.m.)**                         | 6   | 6   | 4   | 7   | 5   | 6   | 3   | 5   | 3   | 2   |
| **Rendimiento neto al final del año (u.m.)** | 4   | 6   | 5   | 8   | 7   | 6   | 3   | 5   | 4   | 2   |


In [15]:
import gurobipy as gp
from gurobipy import GRB
import pandas

#### **Conjuntos**
$$ P: \text{Proyectos} = \{1, 2, 3, 4, 5, 6, 7, 8, 9, 10\} $$

In [16]:
P = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

#### **Parámetros**
$$ I_p: \text{Inversión requerida para el proyecto } p \in P $$
$$ R_p: \text{Rendimiento neto del proyecto } p \in P $$

In [17]:
I = {1: 6, 2: 6, 3: 4, 4: 7, 5: 5, 6: 6, 7: 3, 8: 5, 9: 3, 10: 2}
R = {1: 4, 2: 6, 3: 5, 4: 8, 5: 7, 6: 6, 7: 3, 8: 5, 9: 4, 10: 2}

In [18]:
m = gp.Model("ValoresSA")

#### **Variables de Decisión**
$$ x_p = \begin{cases} 
1 & \text{Si se invierte en el proyecto } p \in P \\ 
0 & \text{d.l.c.} 
\end{cases}  $$

In [19]:
x = m.addVars(P, vtype=GRB.BINARY, name="x")

#### **Función Objetivo**
$$ \text{Maximizar FO} = \sum_{p \in P} R_p x_p $$

In [20]:
m.setObjective(gp.quicksum(R[p] * x[p] for p in P), GRB.MAXIMIZE)

### Restricciones

1. **Restricción de capital disponible:**
$$ \sum_{p \in P} I_p x_p \leq 40 $$

2. **Restricción de número máximo de proyectos:**
$$ \sum_{p \in P} x_p \leq 7 $$

3. **Prohibición de inversión simultánea en proyectos 3, 4 y 7:**
$$ x_3 + x_4 + x_7 \leq 2 $$

4. **Dependencia entre proyectos 3 y 6:**
$$ x_6 \leq x_3 $$

5. **Dependencia entre proyectos 8 y (10 o 4):**
$$ x_8 \leq x_{10} + x_4 $$

6. **Complementariedad entre proyectos 4 y 10:**
$$ x_4 = x_{10} $$


In [21]:
# 1. Restricción de capital disponible
m.addConstr(gp.quicksum(R[p] * x[p] for p in P) <= 40)

# 2. Restricción de número máximo de proyectos
m.addConstr(gp.quicksum(x[p] for p in P) <= 7)

# 3. Prohibición de inversión simultánea en proyectos 3, 4 y 7
m.addConstr(x[3] + x[4] + x[7] <= 2)

# 4. Dependencia entre proyectos 3 y 6
m.addConstr(x[6] <= x[3])

# 5. Dependencia entre proyectos 8 y (10 o 4)
m.addConstr(x[8] <= x[10] + x[4])

# 6. Complementariedad entre proyectos 4 y 10
m.addConstr(x[4] == x[10])

<gurobi.Constr *Awaiting Model Update*>

In [22]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 10 columns and 30 nonzeros
Model fingerprint: 0x30aed5fe
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 4e+01]
Found heuristic solution: objective 35.0000000
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 23 nonzeros
Variable types: 0 continuous, 8 integer (7 binary)

Root relaxation: objective 3.900000e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0   

In [23]:
df_resultados = pandas.DataFrame()
for p in P:
    if x[p].X > 0.9:
        df_resultados.loc[f"Proyecto {p}", "Invierto?"] = "✅"
        df_resultados.loc[f"Proyecto {p}", "Inversión"] = I[p]
        df_resultados.loc[f"Proyecto {p}", "Rendimiento"] = R[p]
    else:
        df_resultados.loc[f"Proyecto {p}", "Invierto?"] = "✖️"
        df_resultados.loc[f"Proyecto {p}", "Inversión"] = 0
        df_resultados.loc[f"Proyecto {p}", "Rendimiento"] = 0
        
df_resultados.loc["Total", "Invierto?"] = ""
df_resultados.loc["Total", "Inversión"] = df_resultados["Inversión"].sum()
df_resultados.loc["Total", "Rendimiento"] = df_resultados["Rendimiento"].sum()
        
df_resultados

,Invierto?,Inversión,Rendimiento
Proyecto 1,✖️,0.0,0.0
Proyecto 2,✅,6.0,6.0
Proyecto 3,✅,4.0,5.0
Proyecto 4,✅,7.0,8.0
Proyecto 5,✅,5.0,7.0
Proyecto 6,✅,6.0,6.0
Proyecto 7,✖️,0.0,0.0
Proyecto 8,✅,5.0,5.0
Proyecto 9,✖️,0.0,0.0
Proyecto 10,✅,2.0,2.0
